In [12]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer

print("Downloading models...")
#Model: DialoGPT - small
tokenizer_dialoGPT_small = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model_dialoGPT_small = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")
print("Done!")

Done!


In [19]:
def generator_dialo_gpt_small(user_input, number_of_candidates):
    # get input
    user_input = user_input
    number_of_candidates = number_of_candidates
    input = tokenizer_dialoGPT_small.encode(user_input, tokenizer_dialoGPT_small.eos_token, return_tensors='pt')
    
    number_of_candidates = int(number_of_candidates)

    beam_outputs = model_dialoGPT_small.generate(
        input,
        max_length=60,
        num_beams=number_of_candidates,
        no_repeat_ngram_size=2,
        num_return_sequences=number_of_candidates,
        early_stopping=True
    )
    #print(beam_outputs)

    beam_output = []
    for step in range(number_of_candidates):
        m = format(tokenizer_dialoGPT_small.decode(beam_outputs[:, input.shape[-1]:][step], skip_special_tokens=True))
        beam_output.append(m)

    return beam_output

In [22]:
candidate = generator_dialo_gpt_small("Where did you go?", 3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [23]:
candidate

['I went to school with him.',
 "I went to school with him. He's a great guy.",
 "I'm going to the game tonight!"]

In [13]:
from transformers import pipeline
fill_mask = pipeline("fill-mask")

In [21]:
fill_mask("I'm going to the <mask> tonight!")

[{'sequence': "I'm going to the movies tonight!",
  'score': 0.13211850821971893,
  'token': 4133,
  'token_str': ' movies'},
 {'sequence': "I'm going to the gym tonight!",
  'score': 0.12617048621177673,
  'token': 6545,
  'token_str': ' gym'},
 {'sequence': "I'm going to the beach tonight!",
  'score': 0.05755433440208435,
  'token': 4105,
  'token_str': ' beach'},
 {'sequence': "I'm going to the pub tonight!",
  'score': 0.04060250148177147,
  'token': 8881,
  'token_str': ' pub'},
 {'sequence': "I'm going to the dentist tonight!",
  'score': 0.026756728067994118,
  'token': 26565,
  'token_str': ' dentist'}]

In [25]:
fill_mask("<mask> went to school with him.")

[{'sequence': 'He went to school with him.',
  'score': 0.05413985252380371,
  'token': 894,
  'token_str': 'He'},
 {'sequence': 'She went to school with him.',
  'score': 0.05307877063751221,
  'token': 2515,
  'token_str': 'She'},
 {'sequence': 'I went to school with him.',
  'score': 0.020075924694538116,
  'token': 100,
  'token_str': 'I'},
 {'sequence': ' never went to school with him.',
  'score': 0.012867623008787632,
  'token': 393,
  'token_str': ' never'},
 {'sequence': ' he went to school with him.',
  'score': 0.012693972326815128,
  'token': 37,
  'token_str': ' he'}]

In [35]:
mask = fill_mask("I went to school with him. He's a <mask> guy.")

In [39]:
for tokens in mask:
    print(tokens['token_str'])

 nice
 funny
 good
 great
 wonderful
